<a href="https://colab.research.google.com/github/jdowner212/cs577_addernet/blob/main/AdderNet_functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import torch

In [7]:
from torch.torch_version import TorchVersion
import numpy as np
import torch

def L1(a,b):
    return -1*np.abs(a-b)

def hard_tanh(value):
    if -1 < value and value < 1:
        return value
    elif value > 1:
        return 1
    elif value < -1:
        return -1

'''Equation 1'''
# modified by Equation 2


'''Equation 2'''
def Y_adder(X, F, m, n, t, similarity_f=L1):
    # F.shape returns (#filters, #channels, #rows, #columns)
    # X.shape returns (#channels, #rows, #columns)
    # t specifies filter #
    sum_ = 0
    _, c_in, d, _ = F.shape     
    for k in range(c_in):
        for j in range(d):
            for i in range(d):
                sum_ += similarity_f(X[k, m+i, n+j], F[i, j, k, t])
    return sum_

'''Equation 3'''
# ignore -- CNN formula

'''Equation 4'''
# ignore -- updated with equation 5

'''Equation 5'''
def dY_dF(X,F,m,n,i,j,k,t):
    X_ = X[k,m+i,n+j]
    F_ = F[t,k,i,j]
    return X_ - F_

'''Equation 6'''
def dY_dX(X,F,m,n,i,j,k,t):
    # clipped, full-precision gradient
    X_ = X[k,m+i,n+j]
    F_ = F[t,k,i,j]
    return hard_tanh(X_ - F_)

'''Equation 7'''
# ignore -- hard_tanh implemented previously

'''Equation 8'''
# ignore -- CNN formula

'''Equation 9'''
def var_Y_adder(X,F,variance_f=torch.var):
    # check torch.var documentation: https://pytorch.org/docs/stable/generated/torch.var.html
    # not sure if we can call torch.var(X) with default parameters
    # or if we need to specify. Does this output a scalar or a tensor?
    var_X = variance_f(X)
    var_F = variance_f(F)
    ###
    _, c_in, d, _ = F.shape
    pi = np.pi

    return np.sqrt(pi/2)*(d**2)*(c_in)*(var_X + var_F)

'''Equation 10'''
def batch_norm(minibatch, gamma, beta):
    m = len(minibatch)
    mean = (1/m)*sum(minibatch)
    std = (1/m)*sum([(x_i-mean)**2 for x_i in minibatch])
    gamma*(minibatch-mean)/std + beta
    return gamma*(minibatch-mean)/std + beta

'''Equation 11'''
def dL_dMinibatch_i(minibatch,dL_dy,i,L,gamma):
    # In dL_dy, y is the result of applying batch_norm to the minibatch
    m = len(minibatch)
    mean = (1/m)*sum(minibatch)
    std = (1/m)*sum([(x_i-mean)**2 for x_i in minibatch])
    
    sum_ = 0
    for j in range(m):
        x_term = (minibatch[i]-minibatch[j])*(minibatch[j]-mean)/std
        sum_ += (dL_dy[i] - dL_dy[j]*(1 + x_term))
    sum_ *= gamma/((m**2)*std)
    
    return sum_

'''Equation 12'''
def delta_F_l(adaptive_lr_l, dL_dF_l, gamma):
    # the update delta for the filter in layer l
    return gamma*adaptive_lr_l*dL_dF_l

'''Equation 13'''
def adaptive_lr_l(dL_dF_l, eta, k):
    # k = number of elements in F_l -- I think equal to len(dL_dF_1)
    # in which case we don't need to explicitly provide it
    l2_norm = torch.sqrt([g**2 for g in dL_dF_l])
    return eta*np.sqrt(k)/l2_norm